- Data source : https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data/data
- Referred Notebook : https://www.kaggle.com/code/akashmathur2212/bitcoin-price-prediction-arima-xgboost-lstm-fbprop#LSTM

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import plotly.express as px
from itertools import product
import warnings
import statsmodels.api as sm

In [4]:
bitstamp = pd.read_csv('/Users/stevencheong/STAT4012/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')

In [5]:
bitstamp.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
bitstamp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4857377 entries, 0 to 4857376
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Timestamp          int64  
 1   Open               float64
 2   High               float64
 3   Low                float64
 4   Close              float64
 5   Volume_(BTC)       float64
 6   Volume_(Currency)  float64
 7   Weighted_Price     float64
dtypes: float64(7), int64(1)
memory usage: 296.5 MB


In [11]:
# Converting the Timestamp column from string to datetime
bitstamp['Timestamp'] = [datetime.fromtimestamp(x) for x in bitstamp['Timestamp']]

In [12]:
print('Dataset Shape: ',  bitstamp.shape)

Dataset Shape:  (4857377, 8)


## Handling missing value

In [13]:
# use interpolate to fill missing value
def fill_missing(df):
    ### function to impute missing values using interpolation ###
    df['Open'] = df['Open'].interpolate()
    df['Close'] = df['Close'].interpolate()
    df['Weighted_Price'] = df['Weighted_Price'].interpolate()

    df['Volume_(BTC)'] = df['Volume_(BTC)'].interpolate()
    df['Volume_(Currency)'] = df['Volume_(Currency)'].interpolate()
    df['High'] = df['High'].interpolate()
    df['Low'] = df['Low'].interpolate()

    print(df.head())
    print(df.isnull().sum())

In [14]:
fill_missing(bitstamp)

            Timestamp  Open  High   Low  Close  Volume_(BTC)  \
0 2011-12-31 15:52:00  4.39  4.39  4.39   4.39      0.455581   
1 2011-12-31 15:53:00  4.39  4.39  4.39   4.39      0.555046   
2 2011-12-31 15:54:00  4.39  4.39  4.39   4.39      0.654511   
3 2011-12-31 15:55:00  4.39  4.39  4.39   4.39      0.753977   
4 2011-12-31 15:56:00  4.39  4.39  4.39   4.39      0.853442   

   Volume_(Currency)  Weighted_Price  
0           2.000000            4.39  
1           2.436653            4.39  
2           2.873305            4.39  
3           3.309958            4.39  
4           3.746611            4.39  
Timestamp            0
Open                 0
High                 0
Low                  0
Close                0
Volume_(BTC)         0
Volume_(Currency)    0
Weighted_Price       0
dtype: int64


In [15]:
bitstamp

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 15:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2011-12-31 15:53:00,4.39,4.39,4.39,4.39,0.555046,2.436653,4.390000
2,2011-12-31 15:54:00,4.39,4.39,4.39,4.39,0.654511,2.873305,4.390000
3,2011-12-31 15:55:00,4.39,4.39,4.39,4.39,0.753977,3.309958,4.390000
4,2011-12-31 15:56:00,4.39,4.39,4.39,4.39,0.853442,3.746611,4.390000
...,...,...,...,...,...,...,...,...
4857372,2021-03-31 07:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,2021-03-31 07:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,2021-03-31 07:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,2021-03-31 07:59:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [25]:
df_csv = bitstamp[(bitstamp['Timestamp'] > '2018-09-25') & (bitstamp['Timestamp'] < '2019-01-02')].reset_index(drop=True)
df_csv

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2018-09-25 00:01:00,6598.88,6601.91,6598.88,6601.75,17.507869,115553.481670,6600.088175
1,2018-09-25 00:02:00,6600.02,6601.17,6600.00,6601.17,0.625429,4128.483922,6601.038358
2,2018-09-25 00:03:00,6601.89,6602.01,6601.32,6602.00,6.713058,44318.766584,6601.874691
3,2018-09-25 00:04:00,6602.65,6604.96,6602.00,6602.00,5.025637,33180.213169,6602.190509
4,2018-09-25 00:05:00,6607.51,6607.82,6606.24,6606.88,2.505021,16551.781741,6607.443549
...,...,...,...,...,...,...,...,...
142554,2019-01-01 23:55:00,3680.50,3680.50,3680.44,3680.44,1.733039,6378.448310,3680.499661
142555,2019-01-01 23:56:00,3681.29,3682.72,3681.29,3681.59,5.296831,19499.287664,3681.312228
142556,2019-01-01 23:57:00,3677.63,3677.63,3677.63,3677.63,0.099500,365.924185,3677.630000
142557,2019-01-01 23:58:00,3680.00,3680.53,3677.20,3677.20,0.318579,1172.446000,3680.234104


In [26]:
df_csv.to_csv('2018-2019_bit.csv')